<a href="https://colab.research.google.com/github/maciejskorski/ml_examples/blob/master/ApproxHessian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Loss Landscape

use random orthogonal directions  https://towardsdatascience.com/visualizing-loss-landscape-of-deep-neural-networks-but-can-we-trust-them-3d3ae0cff46e

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np

import itertools

import seaborn as sn
import matplotlib.pyplot as plt
from matplotlib import cm

In [2]:
## define the LeNet5 architecture

class LeNet5(tf.keras.Model):
  ''' see https://en.wikipedia.org/wiki/Convolutional_neural_network '''

  def __init__(self):
    super(LeNet5, self).__init__()
    self.conv1 = tf.keras.layers.Conv2D(6,kernel_size=(5,5), strides=(1, 1), activation='tanh',padding='valid')
    self.avpool1 = tf.keras.layers.AveragePooling2D(pool_size=(2,2), strides=(2,2), padding='valid')
    self.conv2 = tf.keras.layers.Conv2D(16, kernel_size=(5,5), strides=(1, 1), activation='tanh', padding='valid')
    self.avpool2 = tf.keras.layers.AveragePooling2D(pool_size=(2,2), strides=(2,2), padding='valid')
    self.flatt = tf.keras.layers.Flatten()
    self.dense1 = tf.keras.layers.Dense(120, activation='tanh')
    self.dense2 = tf.keras.layers.Dense(84, activation='tanh')
    self.dense3 = tf.keras.layers.Dense(10, activation='linear')

  def call(self, inputs):
    x = self.conv1(inputs)
    x = self.avpool1(x)
    x = self.conv2(x)
    x = self.avpool2(x)
    x = self.flatt(x)
    x = self.dense1(x)
    x = self.dense2(x)
    x = self.dense3(x)
    return x

class LogisticRegression(tf.keras.Model):
  ''' baseline model '''

  def __init__(self):
    super(LogisticRegression, self).__init__()
    self.flatt = tf.keras.layers.Flatten()
    self.dense1 = tf.keras.layers.Dense(100,activation='relu')
    self.dense2 = tf.keras.layers.Dense(10,activation='linear')

  def call(self, inputs):
    x = self.flatt(inputs)
    x = self.dense1(x)
    x = self.dense2(x)
    return x

In [3]:
## prepare training data
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train/255.0
x_test = x_test/255.0
x_train,x_test = np.expand_dims(x_train,-1),np.expand_dims(x_test,-1) # when black-white

data = tf.data.Dataset.from_tensor_slices((x_train,y_train))
data = data.batch(32).prefetch(1).repeat(1)

In [4]:
## loss and train

model = LeNet5()
optimizer = tf.keras.optimizers.Adam(0.001)

@tf.function
def logp(model,x,y):
  ''' model outputs logits for prediction '''
  y_pred = model(x)
  logp = -tf.keras.losses.sparse_categorical_crossentropy(y,y_pred,from_logits=True)
  return tf.reduce_mean(logp)

@tf.function
def train_step(x,y):
  loss_fn = lambda: -logp(model,x,y)
  vars_fn = lambda: model.trainable_variables
  optimizer.minimize(loss_fn,vars_fn)

In [5]:
## utils

def random_orthogonal_pair(N):
  ''' generate randomly two orthogonal directions '''
  t = np.random.normal(size=(2,N))
  t = t/np.linalg.norm(t,axis=1,keepdims=True)
  a,b = t
  b = b-a*a.dot(b)
  b = b/np.linalg.norm(b)
  return a,b

def model2flatt(model):
  ''' model tensors to one flatt tensor '''
  return np.concatenate([t.numpy().ravel() for t in model.trainable_variables])

def flatt2model(w,shapes):
  ''' one flatt tensor to model tensors '''
  offsets = [np.prod(s) for s in shapes]
  pos_end = list(np.cumsum(offsets))
  pos_start = [0]+pos_end[:-1]
  return [w[i:j].reshape(s) for (i,j,s) in zip(pos_start,pos_end,shapes)]

def loss_on_weights(model,x,y,w):
  ''' loss depending on custom choice of weights '''
  # keep the original weights
  vals_original = [t.numpy() for t in model.trainable_variables]
  # change weigts in the model and call it
  vals = flatt2model(w,[t.shape.as_list() for t in model.trainable_variables])
  for val,var in zip(vals,model.trainable_variables):
    var.assign(val)
  out = logp(model,x,y)
  # restore the original weights
  for val,var in zip(vals_original,model.trainable_variables):
    var.assign(val)
  return out



In [6]:
## train

for i,(x,y) in enumerate(data):
  if i%100 == 0:
    print(logp(model,x_test,y_test).numpy())
  train_step(x,y)

-2.3125286
-0.814244
-0.6945786
-0.6621536
-0.614
-0.5866022
-0.58388
-0.56756926
-0.54807186
-0.5408189
-0.51329786
-0.51388514
-0.51534396
-0.50051004
-0.51413697
-0.4669662
-0.46243623
-0.4804576
-0.46161178


In [ ]:
## compute the loss on weights along two random orthogonal dimensions!

np.random.seed(1234)
N = model.count_params()
a,b=random_orthogonal_pair(N)
w0 = model2flatt(model)

def fn(a_step,b_step):
  return loss_on_weights(model,x_test[:1000],y_test[:1000],w0+a_step*a+b_step*b)

fn = np.vectorize(fn)

xs,ys = 10*np.linspace(-1,1,15),10*np.linspace(-1,1,15)
grid = np.meshgrid(xs,ys,sparse=True)
X,Y,Z = *grid,fn(*grid)
fig, ax = plt.subplots(subplot_kw={"projection": "3d"},figsize=(6,6))
ax.set_xlabel('random direction 1')
ax.set_ylabel('random direction 2')
ax.set_zlabel('loss wrt model params',rotation=90,labelpad=6)
surf = ax.plot_surface(X,Y,-Z, cmap=cm.viridis, linewidth=0, antialiased=False)
plt.show()

In [ ]:
(model(x_test).numpy().argmax(1)==y_test).mean()

## Activations near 0

In [ ]:
sn.set_style("darkgrid")
palette = itertools.cycle(sn.color_palette())

fs = [tf.keras.activations.linear,tf.keras.activations.tanh,tf.keras.activations.sigmoid,tf.keras.activations.relu]
#fs = [np.vectorize(f) for f in fs]

fig,axs = plt.subplots(1,4,figsize=(12,3))

xs = np.linspace(-2,2,100)
for (f,ax) in zip(fs,axs.ravel()):
  ax.plot(xs,f(xs),label=f.__name__,color=next(palette))
  ax.legend()

fig.tight_layout()

## Hessian Approximation

In [ ]:
### utils

## utils for exact hessian form

@tf.function
def logp(logits,y):
  ''' logits of shape (n_batch,n_class), y of shape (n_batch,) '''
  logp = tf.gather(logits,y,batch_dims=1) - tf.reduce_logsumexp(logits,-1) # (n_batch,)
  return tf.reduce_mean(logp) # (,)

@tf.function
def grad(model,var,x,y):
  with tf.GradientTape() as tape:
    logits = model(x)
    loss = -logp(logits,y)
  grad = tape.gradient(loss,var)
  return grad

@tf.function
def hvp(model,var,vec,x,y):
  ''' hessian-vector product; takes advantage of weighted gradient (hess is with respect to weight matrix) '''
  # second derivative (on top)
  with tf.GradientTape() as outer_tape:
    # first derivative (inner)
    with tf.GradientTape() as inner_tape:
      logits = model(x)
      loss = -logp(logits,y)
    grads = inner_tape.gradient(loss,var)
  hess_vec = outer_tape.gradient(grads,var,output_gradients=vec)
  return hess_vec

@tf.function
def hform(model,var,vec,x,y):
  ''' hessian form of model loss, with respect to variable, evaluated at vector '''
  h = hvp(model,var,vec,x,y)
  return tf.reduce_sum(vec*h)

## utils for approx hessian form

@tf.function
def jac_logits_var(model,var,x,y=None):
  ''' jacobian of logits wrt model variable '''
  with tf.GradientTape() as tape:
    logits = model(x)
  jac = tape.jacobian(logits,var)
  return jac

@tf.function
def hess_loss_logits(model,x,y):
  ''' hessian of loss wrt logits '''
  with tf.GradientTape() as outer_tape:
    with tf.GradientTape() as inner_tape:
      logits = model(x)
      loss=-logp(logits,y) # (n_batch,n_class)
    grads = inner_tape.gradient(loss,logits) # (n_batch,n_class)
  return outer_tape.batch_jacobian(grads,logits)  # (n_batch,n_class,n_class)

@tf.function
def hform_approx(model,var,vec,x,y):
  ''' hessian form wrt variable, evaluated on vector; approximate '''
  H = hess_loss_logits(model,x,y) # (n_batch,n_class,n_class)
  J = jac_logits_var(model,var,x) # (n_batch,n_class,var.shape)
  vec = grad(model,var,x,y) # (var.shape,)
  #J_dot_vec = tf.einsum('abcd,cd->ab',J,vec) # (n_batch,n_class)
  axes = [range(2,2+len(vec.shape)),range(len(vec.shape))]
  J_dot_vec = tf.tensordot(J,vec,axes=axes)
  Hform = tf.einsum('xab,xa,xb->',H,J_dot_vec,J_dot_vec)
  return Hform

## utils to refresh initialization

def reset_kernels(model):
  for l in model.layers:
    if hasattr(l,'kernel'):
      l.kernel.assign(l.kernel_initializer(l.kernel.shape))

In [ ]:
## define approximation error

@tf.function
def happrox_error(model,var,x,y):
  vec = grad(model,var,x,y) 
  vec_norm = tf.linalg.norm(vec)
  h_exact = hform(model,var,vec,x,y)
  h_approx = hform_approx(model,var,vec,x,y)
  error = tf.abs(h_approx-h_exact) / tf.square(vec_norm)
  return error

def reset_kernels(model):
  for l in model.layers:
    if hasattr(l,'kernel'):
      l.kernel.assign(l.kernel_initializer(l.kernel.shape))

In [ ]:
import logging
logging.getLogger('tensorflow').disabled = True
import tensorflow as tf
import numpy as np

import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sn
sn.set_style("darkgrid")

## load and normalize data

(x_train,y_train),_ = tf.keras.datasets.cifar10.load_data()
#x_train = x_train.reshape(-1,28*28)
x_train = x_train / 255
x_train = x_train - x_train.mean(0)
y_train = y_train.astype('int32')
x_train = x_train.astype('float32')

In [ ]:
## LeNet-5

activation = tf.keras.activations.tanh

inputs = tf.keras.Input(shape=(32,32,3))

conv1 = tf.keras.layers.Conv2D(filters=6, kernel_size=(3, 3), activation=activation,name='conv1')(inputs)
avpool1 = tf.keras.layers.AveragePooling2D()(conv1)

conv2 = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation=activation,name='conv2')(avpool1)
avpool2 = tf.keras.layers.AveragePooling2D()(conv2)

flatt = tf.keras.layers.Flatten()(avpool2)
dense1 = tf.keras.layers.Dense(units=120, activation=activation,name='dense1')(flatt)
dense2 = tf.keras.layers.Dense(units=84, activation=activation,name='dense2')(dense1)
dense3 = tf.keras.layers.Dense(units=10,name='dense3')(dense2)

logits = dense3
model = tf.keras.Model(inputs=inputs,outputs=logits)

## data

(x_train,y_train),_ = tf.keras.datasets.cifar10.load_data()
x_train = x_train / 255
x_train = x_train - x_train.mean(0)
y_train = y_train.astype('int32').reshape(-1,1)
x_train = x_train.astype('float32')

x,y = x_train[:64],y_train[:64]

## estimate hessian at init

layers = ['conv1','conv2','dense1','dense2','dense3']
diffs = {layer:[] for layer in layers}

for layer in layers:
  
  for _ in range(100):
  
    reset_kernels(model)
    
    var = model.get_layer(layer).kernel
    #var.assign(layer.kernel_initializer(var.shape))
    
    error = happrox_error(model,var,x,y).numpy()

    diffs[layer].append(error)

In [ ]:
fig = plt.figure(constrained_layout=False,figsize=(10,10))
gs = fig.add_gridspec(nrows=2, ncols=6)#, left=0.05, right=0.48, wspace=0.05)
ax1 = fig.add_subplot(gs[0,0:2])
ax2 = fig.add_subplot(gs[0,2:4])
ax3 = fig.add_subplot(gs[0,4:6])
ax4 = fig.add_subplot(gs[1,0:3])
ax5 = fig.add_subplot(gs[1,3:6])
axs = [ax1,ax2,ax3,ax4,ax5]
names = ['dense1','dense2','dense3','conv1','conv2']

for ax,name in zip(axs,names):
  sn.distplot(diffs[name],ax=ax)
  ax.legend([name])

fig.add_subplot(111, frame_on=False)
plt.tick_params(labelcolor='none', top=False, bottom=False, left=False, right=False)
plt.xlabel('relative hessian approximation error at init')
plt.title('LeNet-5 on CIFAR',fontweight="bold")
fig.tight_layout()
plt.show()

## Hessian-driven initialization

In [ ]:
import seaborn as sn

## prepare data
(x_train,y_train),_ = tf.keras.datasets.fashion_mnist.load_data()
x_train = x_train / 255
x_train = x_train - x_train.mean(0)
y_train = y_train.astype('int32')
x_train = x_train.astype('float32')
data = tf.data.Dataset.from_tensor_slices((x_train,y_train.reshape(-1,1))).batch(64).repeat(2)

hforms = []
losses = []

## build model; try different inits

setups = [(tf.keras.activations.relu,tf.keras.initializers.truncated_normal(stddev=0.3),'relu + *hessian driven stddev*'),
          (tf.keras.activations.relu,tf.keras.initializers.he_normal(),'relu + He normal'),
          (tf.keras.activations.tanh,tf.keras.initializers.glorot_uniform(),'tanh + Glorot'),
          (tf.keras.activations.tanh,tf.keras.initializers.orthogonal(),'tanh + Orthogonal'),
          (tf.keras.activations.tanh,tf.keras.initializers.orthogonal(),'relu + Orthogonal')
          ]

#activations = {'sigmoid':tf.keras.activations.sigmoid,'relu':tf.keras.activations.relu,'tanh':tf.keras.activations.tanh}

for setup in setups:

  activation,init,name = setup

  inputs = tf.keras.Input(shape=(28,28,))
  flatt = tf.keras.layers.Flatten()(inputs)
  if name == 'relu + *hessian driven stddev*':
    init1 = 'glorot_uniform'
  else:
    init1 = init
  init2,init3 = init,init
  dense1 = tf.keras.layers.Dense(128,name='dense1',activation=activation,kernel_initializer=init1)(flatt)
  dense2 = tf.keras.layers.Dense(84,name='dense2',activation=activation,kernel_initializer=init2)(dense1)
  dense3 = tf.keras.layers.Dense(10,name='dense3',activation='linear',kernel_initializer=init3)(dense2)
  logits = dense3
  model = tf.keras.Model(inputs=inputs, outputs=logits)

  ## prepare optimizer
  optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
  vars = model.trainable_variables

  layers = ['dense1','dense2','dense3']

  ## estimate hessian at init
  for layer in layers:
    
    for _ in range(100):
    
      reset_kernels(model)
      
      x,y = x_train[:100],y_train[:100]
      var = model.get_layer(layer).kernel
      vec = grad(model,var,x,y)
      
      h = hform(model,var,vec,x,y)
      h = h/tf.linalg.norm(vec)**2

      hforms.append( (name,layer,h.numpy()) )

  ## train

  for i,(x,y) in enumerate(data):
    
    grads = grad(model,vars,x,y)
    optimizer.apply_gradients(zip(grads,vars))

    if i%100 == 0:
      loss = logp(model(x_train),y_train)
      losses.append( (name,i,loss.numpy()) )

In [ ]:
import pandas as pd

hforms = pd.DataFrame(hforms)
hforms.columns = ['setup','layer','init hessian']
losses = pd.DataFrame(losses)
losses.columns = ['setup','step','loss']

sn.set_style("darkgrid")

fig,axs = plt.subplots(len(setups),2,figsize=(10,10))

for (i,setup) in enumerate(setups):
  _,_,s = setup
  title = s
  ax = axs[i][0]
  for l in layers:
    mask = (hforms['setup'] == s) & (hforms['layer'] == l)
    sn.kdeplot(x=hforms[mask]['init hessian'],ax=ax,log_scale=True,label=l)
    ax.set_title(title,fontweight='bold')
    ax.legend()

  mask = losses['setup'] == s
  ax = axs[i][1]
  sn.lineplot(losses[mask]['step'],-losses[mask]['loss'],ax=ax)
  ax.set_title(title,fontweight='bold')

fig.tight_layout()

In [ ]:
step_max = losses['step'].max()
-losses.groupby('setup')[['loss']].max()